In [1]:
import glob
import os
import pandas as pd
import matplotlib.pyplot as plt

from _lammps.get_thermo_from_log import get_lattice_start_end_series_from_thermo_csv
from utils.constants.constants import Constants

root_dir = '/Users/y1u0d2/desktop/Lab/result/lammps/structure-optimization/04'
structures = Constants.structures()

In [2]:
# create all lattice_start_end csv
# for i, structure in enumerate(structures):
#     dirs = glob.glob(f'{root_dir}/{structure}/*')
#     for j, _dir in enumerate(dirs):
#         print(_dir)
#         if i == 0 and j == 0:
#             df = get_lattice_start_end_series_from_thermo_csv(dirs[0])
#             continue
#         tmp_df = get_lattice_start_end_series_from_thermo_csv(_dir)
#         df = pd.concat([df, tmp_df])
# df.to_csv(f'{root_dir}/lattice_s_e.csv', index=False)

In [3]:
structure_df = pd.read_csv(os.path.join(root_dir, 'last_structure_info.csv'))
lattice_df = pd.read_csv(os.path.join(root_dir, 'lattice_s_e.csv'))

In [4]:
# calc volume diff ratio
structure_df['Vol_diff'] = (abs(structure_df.Vol_s - structure_df.Volume) / structure_df.Volume) * 100

In [5]:
def calc_diff_ratio(column, df):
    df[f'{column}_diff'] = (abs(df[column] - df[f'{column}_s']) / df[column]) * 100
    return df
TARGET_COLUMNS = ['CellAlpha', 'CellBeta', 'CellGamma']
for column in TARGET_COLUMNS:
    lattice_df = calc_diff_ratio(column, lattice_df)
lattice_df['Angle_diff_ave'] = (lattice_df['CellAlpha_diff'] + lattice_df['CellBeta_diff'] + lattice_df['CellGamma_diff']) / 3

In [6]:
lattice_df_ave = lattice_df.copy()
for column in lattice_df_ave.columns:
    if 'Cell' in column:
        lattice_df_ave.drop(column, axis=1 ,inplace=True)
lattice_df_ave

,structure,idx,Angle_diff_ave
0,alpha-critobalite,46,0.0
1,alpha-critobalite,666,0.0
2,alpha-critobalite,840,0.0
3,alpha-critobalite,894,0.0
4,alpha-critobalite,1056,0.0
...,...,...,...
74,stishovite,46,0.0
75,stishovite,128,0.0
76,stishovite,670,0.0
77,stishovite,659,0.0


In [7]:
df = pd.merge(structure_df, lattice_df_ave, on=['structure', 'idx'])

In [8]:
max_df = df.groupby('structure').max()
min_df = df.groupby('structure').min()

In [9]:
max_df = max_df.reset_index()
max_df.drop(['idx','Vol_s','natom','TotEng','Temp','Lx','Volume','Eng_a'], axis=1, inplace=True)
max_df.rename(columns={
    'Vol_diff':'Vol_diff_max',
    'Angle_diff_ave':'Angle_diff_ave_max'
}, inplace=True)

min_df = min_df.reset_index()
min_df.drop(['idx','Vol_s','natom','TotEng','Temp','Lx','Volume','Eng_a'], axis=1, inplace=True)
min_df.rename(columns={
    'Vol_diff':'Vol_diff_min',
    'Angle_diff_ave':'Angle_diff_ave_min',
}, inplace=True)
min_max_df = pd.merge(max_df, min_df, on='structure')

In [10]:
min_max_df.to_csv(os.path.join(root_dir,'min_max.csv'), index=False)

In [13]:
all_in_df = pd.merge(structure_df, lattice_df, on=['idx', 'structure'])

In [14]:
all_in_df.groupby

,structure,idx,Vol_s,natom,TotEng,Temp,Lx,Volume,Eng_a,Vol_diff,...,Cella_s,Cellb_s,Cellc_s,CellAlpha_s,CellBeta_s,CellGamma_s,CellAlpha_diff,CellBeta_diff,CellGamma_diff,Angle_diff_ave
0,alpha-critobalite,46,839.05543,48,-27524.045,0,10.146854,738.42174,-573.4176,13.628213,...,10.588314,10.588314,7.484056,90.0,90.0,90.0,0.0,0.0,0.0,0.0
1,alpha-critobalite,666,858.95660,48,-27523.579,0,10.445963,805.66794,-573.4079,6.614221,...,10.671370,10.671370,7.542770,90.0,90.0,90.0,0.0,0.0,0.0,0.0
2,alpha-critobalite,840,980.63842,48,-27524.087,0,10.134698,736.08170,-573.4185,33.224127,...,11.151632,11.151632,7.885551,90.0,90.0,90.0,0.0,0.0,0.0,0.0
3,alpha-critobalite,894,881.57394,48,-27524.143,0,10.090845,726.40068,-573.4196,21.361938,...,10.763538,10.763538,7.609369,90.0,90.0,90.0,0.0,0.0,0.0,0.0
4,alpha-critobalite,1056,562.08280,48,-27524.092,0,9.901927,657.56220,-573.4186,14.520208,...,9.397396,9.397396,6.364807,90.0,90.0,90.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,stishovite,46,375.99897,48,-27524.847,0,8.328915,369.07808,-573.4343,1.875183,...,8.380654,8.380654,5.353414,90.0,90.0,90.0,0.0,0.0,0.0,0.0
75,stishovite,128,361.81364,48,-27524.848,0,8.301646,368.14179,-573.4343,1.718944,...,8.253804,8.253804,5.311004,90.0,90.0,90.0,0.0,0.0,0.0,0.0
76,stishovite,670,331.97743,48,-27524.756,0,8.264810,366.91403,-573.4324,9.521740,...,7.993696,7.993696,5.195332,90.0,90.0,90.0,0.0,0.0,0.0,0.0
77,stishovite,659,331.97743,48,-27524.756,0,8.265356,366.98672,-573.4324,9.539661,...,7.993696,7.993696,5.195332,90.0,90.0,90.0,0.0,0.0,0.0,0.0
